In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Conv1D,MaxPooling1D, Flatten

from keras.utils import multi_gpu_model
from keras import backend as K
from sklearn.cross_validation import train_test_split
import matplotlib
matplotlib.use('agg')
import pylab as plt
import pandas as pd
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import Normalizer
from keras.utils import np_utils

import h5py

X_train = np.load("20170827_mawilab_flow_000features.pkl")
Y_train = np.load("20170827_mawilab_flow_000labels.pkl")

scaler = Normalizer().fit(X_train)
X_train = scaler.transform(X_train)
scaler = Normalizer().fit(Y_train)
Y_train = scaler.transform(Y_train)

# Test Set 1 : 20170827_mawilab_flow_001
X_test1 = np.load("20170827_mawilab_flow_001features.pkl")
Y_test1 = np.load("20170827_mawilab_flow_001labels.pkl")


# Test Set 2 : 20170827_mawilab_flow_002
X_test2 = np.load("20170827_mawilab_flow_002features.pkl")
Y_test2 = np.load("20170827_mawilab_flow_002labels.pkl")


# Test Set 3 : 20170827_mawilab_flow_003
X_test3 = np.load("20170827_mawilab_flow_003features.pkl")
Y_test3 = np.load("20170827_mawilab_flow_003labels.pkl")


# Test Set 4: 20170827_mawilab_flow_004
X_test4 = np.load("20170827_mawilab_flow_004features.pkl")
Y_test4 = np.load("20170827_mawilab_flow_004labels.pkl")

# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test1 = np.reshape(X_test1, (X_test1.shape[0],X_test1.shape[1],1))
X_test2 = np.reshape(X_test2, (X_test2.shape[0],X_test2.shape[1],1))
X_test3 = np.reshape(X_test3, (X_test3.shape[0],X_test3.shape[1],1))
X_test4 = np.reshape(X_test4, (X_test4.shape[0],X_test4.shape[1],1))


C:\Anaconda\envs\tensorflow-cpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Anaconda\envs\tensorflow-cpu\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
cnn1d_1 = Sequential([Conv1D(64, 3, padding="same",input_shape=(16, 1)),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(2),
    Activation('sigmoid'),
])
print(cnn1d_1.summary())

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 16, 64)            256       
_________________________________________________________________
activation_1 (Activation)    (None, 16, 64)            0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 8, 64)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               65664     
_________________________________________________________________
activation_2 (Activation)    (None, 128)               0         
___________________________________________________________

In [3]:
#cnn1d_1 = multi_gpu_model(cnn1d_1, gpus=8)
cnn1d_1.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
# train
start_time = time.time()
history = cnn1d_1.fit(X_train, Y_train, batch_size=64, validation_data=(X_test4, Y_test4) ,epochs=25)
end_time = time.time() 
print("Total time taken to train the Training model is", (end_time - start_time))
# serialize model to JSON
model_json = cnn1d_1.to_json()
with open("20170827_mawilab_flow_000.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cnn1d_1.save_weights("20170827_mawilab_flow_000.h5")
print("Saved model to disk")
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('Acc_Valaccuaracy00_04.png')

Train on 38361 samples, validate on 42599 samples
Epoch 1/25
38361/38361 [==============================] - 3s 75us/step - loss: 0.5290 - acc: 0.6997 - val_loss: 4.1738 - val_acc: 0.7166
Epoch 2/25
38361/38361 [==============================] - 2s 60us/step - loss: 0.4740 - acc: 0.7143 - val_loss: 4.2965 - val_acc: 0.7254
Epoch 3/25
38361/38361 [==============================] - 2s 59us/step - loss: 0.4677 - acc: 0.7244 - val_loss: 4.3399 - val_acc: 0.7236
Epoch 4/25
38361/38361 [==============================] - 2s 59us/step - loss: 0.4620 - acc: 0.7272 - val_loss: 4.0378 - val_acc: 0.7422
Epoch 5/25
38361/38361 [==============================] - 2s 59us/step - loss: 0.4577 - acc: 0.7302 - val_loss: 4.3149 - val_acc: 0.7221
Epoch 6/25
38361/38361 [==============================] - 2s 59us/step - loss: 0.4519 - acc: 0.7333 - val_loss: 4.1453 - val_acc: 0.7407
Epoch 7/25
38361/38361 [==============================] - 2s 59us/step - loss: 0.4475 - acc: 0.7358 - val_loss: 4.1701 - val_acc